In [284]:
import pandas as pd
from tqdm import tqdm
pd.options.mode.chained_assignment = None  # default='warn'

In [330]:
def format_df(df): 
    df = df.iloc[1:].reset_index(drop=True)
    df = df.append(df.iloc[0], ignore_index=True)

    list_index = list(df[df['Time Interval'].str.contains("JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC")].index)
    list_index += [len(df) - 1]

    counter = 0
    list_acc = []
    list_ = []
    for i in range(0, len(df)): 
        try:
            if i < list_index[counter]: 
                list_.append(i)
            else: 
                counter += 1
                list_acc.append(list_)
                list_ = []
        except:
            break

    date_list = list(df.iloc[list_index]['Time Interval'])
    list_acc = list_acc[1:]
    lists = []
    for i in range(len(list_acc)): 
        list2 = []
        list_ = list_acc[i]
        date = date_list[i]
        for i in list_: 
            list2.append(date)
        lists.append(list2)

    list_subsets = [df.iloc[list_] for list_ in list_acc]

    subsets = []
    for i in range(len(list_subsets)):
        subset = list_subsets[i]
        subset['date'] = date_list[i]
        subsets.append(subset)

    df_f1 = pd.concat(subsets)
    df_f1['start_full_time'] = pd.to_datetime(df_f1['date'].apply(lambda x: str(x)[:9]) + df_f1['Time Interval'].apply(lambda x: ' ' + str(x)[:5]))
    df_f1['start_full_time'] = df_f1['start_full_time'].dt.tz_localize('US/Eastern').dt.tz_convert('Japan')
    df_f1['start_date'] = df_f1['start_full_time'].dt.date
    df_f1['start_time_int'] = df_f1['start_full_time'].dt.time
    
    return df_f1

In [331]:
file_names = ['2010', '2010-11a', '2011-12', '2011a-11b', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21']
file_names = ['raw_data/topix/' + name + '.xlsx' for name in file_names]

In [332]:
df_list = [pd.read_excel(file_name) for file_name in file_names]

In [333]:
topix_df = pd.concat([format_df(df) for df in tqdm(df_list)])


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:09<00:00,  1.36it/s]


In [340]:
topix_df.to_pickle('checkpoint_data/topix_df.pkl')